In [7]:
import pandas as pd
import sys
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from model.model_create import *
from preprocessing.preprocess_1_cleaning import *
import pickle


In [134]:
raw_data = pd.read_csv('../raw_data/reference_csv_v1')
# api_data = pd.read_csv('../raw_data/')

In [136]:
raw_data[['genres', 'platforms']] = make_list_columns_to_lists(raw_data, ['genres', 'platforms'])

In [140]:
type(raw_data['platforms'][0])

list

In [48]:
raw_data['platforms'] = make_list_columns_to_lists(raw_data, ['platforms'])

In [108]:
genre_df, mlb = categorical_encoder(raw_data['genres'])

In [111]:
platform_df, mlb = categorical_encoder(raw_data['platforms'])

In [116]:
platform_list = list(platform_df.columns)

In [133]:
platform_df.sum().sort_values(ascending=False)[20:30].index.to_list

<bound method IndexOpsMixin.tolist of Index(['PlayStation Vita', 'Nintendo 3DS', 'Xbox', 'Wii U',
       'Sega Mega Drive/Genesis', 'NES', 'Game Boy Advance', 'SNES',
       'Nintendo GameCube', 'Amiga'],
      dtype='object')>

In [143]:
from joblib import dump, load
from json import loads, dumps

In [144]:
games = pd.read_json('../raw_data/reference_df_v1', orient='records', lines=True)
# X_train = pd.read_csv('raw_data/basline_X_train')
X_train = pd.read_json('../raw_data/xtrain_df_v1', orient='records', lines=True)
# model = pickle.load(open('raw_data/basline_model', 'rb'))
model = load('../raw_data/model_v1.joblib')

/Users/antonis/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KNeighborsRegressor from version 1.2.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [154]:
datetime.datetime.utcfromtimestamp(games['release_date'][0])

ValueError: year 56969 is out of range

In [149]:
def unix_to_timestamp(unix_timestamp):
    timestamp = datetime.datetime.utcfromtimestamp(unix_timestamp)
    return timestamp.strftime('%Y-%m-%d %H:%M:%S UTC')

In [152]:
games['release_date'][0]

1735603200000

In [60]:
type(raw_data['release_date'][0])

str

In [61]:
raw_data_date = change_to_datetype(raw_data, 'release_date')

In [64]:
raw_data_date['release_date'][0]

Timestamp('2024-12-31 00:00:00')

In [67]:
zone_1 = pd.Timestamp(datetime.datetime.strptime('2000-01-01', '%Y-%m-%d'))
zone_2 = pd.Timestamp(datetime.datetime.strptime('2010-01-01', '%Y-%m-%d'))
zone_3 = pd.Timestamp(datetime.datetime.strptime('2020-01-01', '%Y-%m-%d'))
today_date = pd.Timestamp(datetime.date.today())

raw_data_date['release_date'][0] > today_date

True

In [93]:
def time_range_start_stop(df, start, stop):
    zone_1 = pd.Timestamp(datetime.datetime.strptime('1980-01-01', '%Y-%m-%d'))
    zone_2 = pd.Timestamp(datetime.datetime.strptime('1990-01-01', '%Y-%m-%d'))
    zone_3 = pd.Timestamp(datetime.datetime.strptime('2000-01-01', '%Y-%m-%d'))
    zone_4 = pd.Timestamp(datetime.datetime.strptime('2010-01-01', '%Y-%m-%d'))
    zone_5 = pd.Timestamp(datetime.datetime.strptime('2020-01-01', '%Y-%m-%d'))
    today_date = pd.Timestamp(datetime.date.today())

    
    if start == 'Prehistoric':
        df_filt_start = df[df['release_date'].apply(lambda x: 
                    zone_1 > x)]
    if start == '1980s':
        df_filt_start = df[df['release_date'].apply(lambda x: 
                    zone_2 < x)]
    if start == '1990s':
        df_filt_start = df[df['release_date'].apply(lambda x: 
                    zone_3 < x)]
    if start == '2000s':
        df_filt_start = df[df['release_date'].apply(lambda x: 
                    zone_4 < x)]
    if start == '2010s':
        df_filt_start = df[df['release_date'].apply(lambda x: 
                    zone_5 < x)]
    if start == '2020s':
        df_filt_start = df[df['release_date'].apply(lambda x: 
                    today_date < x)]
    if start == 'Upcoming':
        df_filt_start = df[df['release_date'].apply(lambda x: 
                    today_date > x)]
    
    if stop == 'Prehistoric':
        return df_filt_start
    if stop == '1980s':
        df_filt_end = df_filt_start[df_filt_start['release_date'].apply(lambda x:
                    zone_2 > x)]
        return df_filt_end
    if stop == '1990s':
        df_filt_end = df_filt_start[df_filt_start['release_date'].apply(lambda x:
                    zone_3 > x)]
        return df_filt_end
    if stop == '2000s':
        df_filt_end = df_filt_start[df_filt_start['release_date'].apply(lambda x:
                    zone_4 > x)]
        return df_filt_end
    if stop == '2010s':
        df_filt_end = df_filt_start[df_filt_start['release_date'].apply(lambda x:
                    zone_5 > x)]
        return df_filt_end
    if stop == '2020s':
        df_filt_end = df_filt_start[df_filt_start['release_date'].apply(lambda x:
                    today_date > x)]
        return df_filt_end
    if stop == 'Upcoming':
        return df_filt_start

In [96]:
time_range_start_stop(raw_data_date, '2000s', 'Upcoming')

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3,link,game_id
0,Darling Duality: Winter Wish,2024-12-31,7,0,2,2,['Melancholy Marionette'],2.2,[Visual Novel],"[Windows PC, Mac, Linux]",...,0,0,0,0,0,0,0,1,/games/darling-duality-winter-wish/,178780.0
1,Anger Foot,2024-12-31,80,1,82,297,"['Devolver Digital', 'Robbie Fraser']",3.3,"[Brawler, Indie, Shooter]",[Windows PC],...,1,0,0,0,0,0,0,1,/games/anger-foot/,147184.0
2,Life Is Feudal: MMO,2023-12-31,9,0,5,1,['Long Tale Games'],2.4,"[Adventure, RPG, Simulator]",[Windows PC],...,0,0,0,0,0,0,0,1,/games/life-is-feudal-mmo/,24611.0
3,The Making of Karateka,2023-08-29,10,3,9,41,['Digital Eclipse'],4.5,"[Adventure, Fighting, Platform]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,0,0,1,0,0,0,0,1,/games/the-making-of-karateka/,256293.0
4,Enigma of Fear,2023-12-31,16,0,26,155,['Dumativa Game Studio'],2.1,"[Adventure, Indie, Puzzle]",[Windows PC],...,0,0,0,0,0,0,0,1,/games/enigma-of-fear/,140133.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36843,The Smurfs' Village,2010-11-10,19,1,1,1,[''],2.2,[],[iOS],...,0,0,0,0,0,0,1,0,/games/the-smurfs-village/,109159.0
36844,Davigo,2023-12-31,4,0,2,8,['Davigo Studio'],3.5,[Indie],"[SteamVR, Oculus Quest 2]",...,0,0,0,0,0,0,0,1,/games/davigo/,121403.0
36847,Sea Battle,2014-02-07,3,0,0,0,['BYRIL'],2.2,[],"[Android, iOS]",...,0,0,0,0,0,0,0,1,/games/sea-battle--2/,134381.0
36848,We Are Illuminati,2017-11-02,18,0,0,0,['Tapps Games'],2.0,"[Indie, Simulator, Strategy]","[Android, iOS]",...,0,1,0,0,0,0,0,1,/games/we-are-illuminati/,120333.0


In [76]:
def time_range(df, range):
    zone_1 = pd.Timestamp(datetime.datetime.strptime('1980-01-01', '%Y-%m-%d'))
    zone_2 = pd.Timestamp(datetime.datetime.strptime('1990-01-01', '%Y-%m-%d'))
    zone_3 = pd.Timestamp(datetime.datetime.strptime('2000-01-01', '%Y-%m-%d'))
    zone_4 = pd.Timestamp(datetime.datetime.strptime('2010-01-01', '%Y-%m-%d'))
    zone_5 = pd.Timestamp(datetime.datetime.strptime('2020-01-01', '%Y-%m-%d'))
    today_date = pd.Timestamp(datetime.date.today())

    
    if range == 'Anytime':
        return df
    else:
        if range == 'Prehistoric':
            df_filt = df[df['release_date'].apply(lambda x: 
                    zone_1 > x)]
            return df_filt
        if range == "1980s":
            df_filt = df[df['release_date'].apply(lambda x: 
                    zone_1 < x and zone_2 > x)]
            return df_filt
        if range == "1990s":
            df_filt = df[df['release_date'].apply(lambda x: 
                    zone_2 < x and zone_3 > x)]
            return df_filt   
        if range == "2000s":
            df_filt = df[df['release_date'].apply(lambda x: 
                    zone_3 < x and zone_4 > x)]
            return df_filt   
        if range == "2010s":
            df_filt = df[df['release_date'].apply(lambda x: 
                    zone_4 < x and zone_5 > x)]
            return df_filt  
        if range == "2020s":
            df_filt = df[df['release_date'].apply(lambda x: 
                    zone_5 < x and today_date > x)]
            return df_filt
        if range == "Upcoming":
            df_filt = df[df['release_date'].apply(lambda x: 
                    today_date < x)]
            return df_filt

In [78]:
time_range(raw_data_date, "1980's")
    

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3,link,game_id
30543,Frogger,1981-12-31,6,0,1,1,[''],1.7,[],[],...,0,0,0,0,0,1,0,0,/games/frogger--3/,198743.0
30544,Frogger,1981-12-31,6,0,1,1,[''],1.7,[],[],...,0,0,0,0,0,1,0,0,/games/frogger--3/,18288.0
34986,Pinball Quest,1989-12-15,31,0,16,4,"['TOSE Co.', 'Ltd.', 'Jaleco Ltd.']",2.6,"[Adventure, Pinball]",[NES],...,0,0,0,0,0,1,0,0,/games/pinball-quest/,48026.0
35032,Ys II,1988-06-24,69,3,39,22,"['Nihon Falcom', 'XSEED Games']",3.0,"[Adventure, RPG]","[MSX2, PC-8801, PC DOS, FM-7, Family Computer ...",...,0,0,0,0,0,1,0,0,/games/ys-ii/,206537.0
35033,Ys II,1988-06-24,69,3,39,22,"['Nihon Falcom', 'XSEED Games']",3.0,"[Adventure, RPG]","[MSX2, PC-8801, PC DOS, FM-7, Family Computer ...",...,0,0,0,0,0,1,0,0,/games/ys-ii/,90549.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36727,Moon Cresta,1980-02-15,47,0,17,11,"['Nichibutsu', 'Incentive Software']",2.7,"[Arcade, Shooter]","[Dragon 32/64, Arcade, Wii, BBC Microcomputer ...",...,1,0,0,1,0,1,0,0,/games/moon-cresta/,18762.0
36728,Crazy Balloon,1980-02-01,11,0,2,1,['Taito Corporation'],2.3,[Puzzle],[Arcade],...,0,0,0,0,0,1,0,0,/games/crazy-balloon/,38532.0
36729,Dragster,1980-02-01,64,1,10,4,['Activision'],2.1,[Racing],[Atari 2600],...,0,0,0,0,0,1,0,0,/games/dragster/,46885.0
36730,Cosmic Alien,1980-02-01,4,0,1,1,[''],2.7,[Shooter],[Arcade],...,1,0,0,0,0,1,0,0,/games/cosmic-alien/,40021.0


In [54]:
plat_sel = ['Windows PC', 'Xbox One']

def platform_filter(df, selected_platforms):
    if selected_platforms == ['Any']:
        pass
    else:
        if len(selected_platforms) == 1:
            df_filt = df[df['platforms'].apply(lambda x: 
                selected_platforms[0] in x)]
            return df_filt
        if len(selected_platforms) == 2:
            df_filt = df[df['platforms'].apply(lambda x: 
                selected_platforms[0] in x or 
                selected_platforms[1] in x)]
            return df_filt
        if len(selected_platforms) == 3:
            df_filt = df[df['platforms'].apply(lambda x: 
                selected_platforms[0] in x or 
                selected_platforms[1] in x or
                selected_platforms[2] in x)]
            return df_filt
        if len(selected_platforms) == 4:
            df_filt = df[df['platforms'].apply(lambda x: 
                selected_platforms[0] in x or 
                selected_platforms[1] in x or
                selected_platforms[2] in x or
                selected_platforms[3] in x)]
            return df_filt
        if len(selected_platforms) == 5:
            df_filt = df[df['platforms'].apply(lambda x: 
                selected_platforms[0] in x or 
                selected_platforms[1] in x or
                selected_platforms[2] in x or
                selected_platforms[3] in x or 
                selected_platforms[4] in x)]
            return df_filt

In [55]:
platform_filter(raw_data, plat_sel)

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3,link,game_id
0,Darling Duality: Winter Wish,2024-12-31,7,0,2,2,['Melancholy Marionette'],2.2,[Visual Novel],"[Windows PC, Mac, Linux]",...,0,0,0,0,0,0,0,1,/games/darling-duality-winter-wish/,178780.0
1,Anger Foot,2024-12-31,80,1,82,297,"['Devolver Digital', 'Robbie Fraser']",3.3,"[Brawler, Indie, Shooter]",[Windows PC],...,1,0,0,0,0,0,0,1,/games/anger-foot/,147184.0
2,Life Is Feudal: MMO,2023-12-31,9,0,5,1,['Long Tale Games'],2.4,"[Adventure, RPG, Simulator]",[Windows PC],...,0,0,0,0,0,0,0,1,/games/life-is-feudal-mmo/,24611.0
3,The Making of Karateka,2023-08-29,10,3,9,41,['Digital Eclipse'],4.5,"[Adventure, Fighting, Platform]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,0,0,1,0,0,0,0,1,/games/the-making-of-karateka/,256293.0
4,Enigma of Fear,2023-12-31,16,0,26,155,['Dumativa Game Studio'],2.1,"[Adventure, Indie, Puzzle]",[Windows PC],...,0,0,0,0,0,0,0,1,/games/enigma-of-fear/,140133.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36835,OXO,1952-12-31,34,0,8,8,"['Alexander Shafto ""Sandy"" Douglas', 'Universi...",3.0,"[Puzzle, Strategy]","[Windows PC, EDSAC]",...,0,1,0,0,0,1,0,0,/games/oxo/,11323.0
36839,Skator Gator 3D,2023-08-31,8,1,5,21,['Weathered Sweater'],3.4,"[Adventure, Indie, Sport]",[Windows PC],...,0,0,0,0,1,0,0,1,/games/skator-gator-3d/,217239.0
36841,Small Saga,2023-11-16,4,0,9,38,['Jeremy Noghani'],3.7,"[Adventure, Indie, RPG]","[Windows PC, Mac]",...,0,0,0,0,0,0,0,1,/games/small-saga/,119278.0
36842,A Monster's Insight,2021-04-02,4,0,5,0,[''],3.8,[Visual Novel],[Windows PC],...,0,0,0,0,0,0,0,1,/games/a-monsters-insight/,178813.0


In [19]:
'RPG' in raw_data['genres'][2]

True

In [20]:
filtered_df = raw_data[raw_data['genres'].apply(lambda x: 'RPG' in x)]

In [43]:
gen_sel = ['RPG', 'Indie', 'Shooter']

def genre_filter(df, selected_genres):
    if selected_genres == ['Any']:
        pass
    else:
        if len(selected_genres) == 1:
            df_filt = df[df['genres'].apply(lambda x: 
                selected_genres[0] in x)]
            return df_filt
        if len(selected_genres) == 2:
            df_filt = df[df['genres'].apply(lambda x: 
                selected_genres[0] in x or 
                selected_genres[1] in x)]
            return df_filt
        if len(selected_genres) == 3:
            df_filt = df[df['genres'].apply(lambda x: 
                selected_genres[0] in x or 
                selected_genres[1] in x or
                selected_genres[2] in x)]
            return df_filt
        if len(selected_genres) == 4:
            df_filt = df[df['genres'].apply(lambda x: 
                selected_genres[0] in x or 
                selected_genres[1] in x or
                selected_genres[2] in x or
                selected_genres[3] in x)]
            return df_filt
        if len(selected_genres) == 5:
            df_filt = df[df['genres'].apply(lambda x: 
                selected_genres[0] in x or 
                selected_genres[1] in x or
                selected_genres[2] in x or
                selected_genres[3] in x or 
                selected_genres[4] in x)]
            return df_filt

In [44]:
genre_filter(raw_data, gen_sel)

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3,link,game_id
1,Anger Foot,2024-12-31,80,1,82,297,"['Devolver Digital', 'Robbie Fraser']",3.3,"[Brawler, Indie, Shooter]",['Windows PC'],...,1,0,0,0,0,0,0,1,/games/anger-foot/,147184.0
2,Life Is Feudal: MMO,2023-12-31,9,0,5,1,['Long Tale Games'],2.4,"[Adventure, RPG, Simulator]",['Windows PC'],...,0,0,0,0,0,0,0,1,/games/life-is-feudal-mmo/,24611.0
4,Enigma of Fear,2023-12-31,16,0,26,155,['Dumativa Game Studio'],2.1,"[Adventure, Indie, Puzzle]",['Windows PC'],...,0,0,0,0,0,0,0,1,/games/enigma-of-fear/,140133.0
5,They Speak From the Abyss,2023-12-31,10,0,48,85,"['2 Left Thumbs', 'Nikki Kalpa']",3.2,"[Adventure, Indie, RPG]",['Windows PC'],...,0,0,0,0,0,0,0,1,/games/they-speak-from-the-abyss/,153550.0
6,Shred and Tear: Explosive Kajun,2023-12-31,3,0,0,9,"['Shady Corner Games', 'Lucid Realm Games']",1.5,"[Brawler, Indie]",['Windows PC'],...,0,0,0,0,0,0,0,1,/games/shred-and-tear-explosive-kajun/,192702.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36844,Davigo,2023-12-31,4,0,2,8,['Davigo Studio'],3.5,[Indie],"['SteamVR', 'Oculus Quest 2']",...,0,0,0,0,0,0,0,1,/games/davigo/,121403.0
36845,Final Crisis: Terrestrial Defense Police,1991-12-01,4,0,1,4,['Techno Grard'],3.8,[Shooter],['PC-8801'],...,1,0,0,0,0,1,0,0,/games/final-crisis/,141644.0
36846,Level Up!,2009-11-25,6,0,0,1,[''],2.7,"[Adventure, Indie]",['Web browser'],...,0,0,0,0,0,0,1,0,/games/level-up--1/,95373.0
36848,We Are Illuminati,2017-11-02,18,0,0,0,['Tapps Games'],2.0,"[Indie, Simulator, Strategy]","['Android', 'iOS']",...,0,1,0,0,0,0,0,1,/games/we-are-illuminati/,120333.0


In [10]:
model_new, ref_new, x_train_new = create_baseline_model_new_csv(raw_data=raw_data) 

ValueError: could not convert string to float: '/games/darling-duality-winter-wish/'

In [4]:
with open('../model/basline_model_v2', 'wb') as file:
    pickle.dump(model_new, file)

In [5]:
ref_new.to_csv('../raw_data/baseline_reference_v2', index=False)
x_train_new.to_csv('../raw_data/basline_X_train_v2', index=False)

In [7]:
raw_data

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,Simulator,Puzzle,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3
0,Darling Duality: Winter Wish,2024-12-31,7,0,2,2,['Melancholy Marionette'],2.2,['Visual Novel'],"['Windows PC', 'Mac', 'Linux']",...,0,0,0,0,0,0,0,0,0,1
1,Anger Foot,2024-12-31,80,1,82,297,"['Devolver Digital', 'Robbie Fraser']",3.3,"['Brawler', 'Indie', 'Shooter']",['Windows PC'],...,0,0,1,0,0,0,0,0,0,1
2,Life Is Feudal: MMO,2023-12-31,9,0,5,1,['Long Tale Games'],2.4,"['Adventure', 'RPG', 'Simulator']",['Windows PC'],...,1,0,0,0,0,0,0,0,0,1
3,The Making of Karateka,2023-08-29,10,3,9,41,['Digital Eclipse'],4.5,"['Adventure', 'Fighting', 'Platform']","['Windows PC', 'PlayStation 4', 'Xbox One', 'P...",...,0,0,0,0,1,0,0,0,0,1
4,Enigma of Fear,2023-12-31,16,0,26,155,['Dumativa Game Studio'],2.1,"['Adventure', 'Indie', 'Puzzle']",['Windows PC'],...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36815,Final Crisis: Terrestrial Defense Police,1991-12-01,4,0,1,4,['Techno Grard'],3.8,['Shooter'],['PC-8801'],...,0,0,1,0,0,0,0,1,0,0
36816,Level Up!,2009-11-25,6,0,0,1,[''],2.7,"['Adventure', 'Indie']",['Web browser'],...,0,0,0,0,0,0,0,0,1,0
36817,Sea Battle,2014-02-07,3,0,0,0,['BYRIL'],2.2,[''],"['Android', 'iOS']",...,0,0,0,0,0,0,0,0,0,1
36818,We Are Illuminati,2017-11-02,18,0,0,0,['Tapps Games'],2.0,"['Indie', 'Simulator', 'Strategy']","['Android', 'iOS']",...,1,0,0,1,0,0,0,0,0,1


In [8]:
links_df = pd.read_csv('../raw_data/all_links_with_game_id_v1')

In [11]:
def remove_link_trail(string):
    update = string.replace('https://www.backloggd.com', '')
    return update

In [12]:
refernce_data['url'] = refernce_data['url'].apply(remove_link_trail)

In [16]:
links_df.shape

(120096, 2)

In [17]:
links_df.drop_duplicates(inplace=True)

In [28]:
merged = refernce_data.merge(links_df, left_on='url', right_on='link', how='left')

In [36]:
mask = merged['title'] == 'Omori'
mask_isna = merged['game_id'].isna()

missing_ids = merged[mask_isna]['url']

In [51]:
missing_ids

1                               /games/anger-foot/
4                           /games/enigma-of-fear/
7              /games/inazuma-eleven-victory-road/
19                    /games/goodbye-volcano-high/
51                   /games/disparity-of-the-dead/
                           ...                    
36834                          /games/draughts--1/
36835                     /games/bertie-the-brain/
36836    /games/cathode-ray-tube-amusement-device/
36840                   /games/a-monsters-insight/
36843                         /games/final-crisis/
Name: url, Length: 2425, dtype: object

In [24]:
mask2 = refernce_data['title'] == 'Omori'

refernce_data[mask2]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,Simulator,Puzzle,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3
4532,Omori,2020-12-25,11000,1300,6400,5000,"[OMOCAT, PLAYISM]",4.1,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, Mac, PlayStation 4, Xbox One, Nin...",...,0,0,0,0,0,0,0,0,0,1


In [53]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import pickle
import time
import csv

# Prepeing final list and setting count for montoring progress
missed_data = []
game_data = []

for game in missing_ids:
    try:
        # Setting up the html parser + beautful soup
        game_response = requests.get(f"https://www.backloggd.com{game}")
        game_soup = BeautifulSoup(game_response.content, 'html.parser')

        # Scraping the title
        title = game_soup.find('h1', class_='mb-0').string
        game_id = game_soup.find('div', class_ = 'card mx-auto game-cover overlay-hide')['game_id']

        game_data.append({'title': title,
                        'url':game, 
                        'game_id':game_id})
    except:
        print(f'Failed at {game}')
        missed_data.append(game)
        time.sleep(10)

game_df = pd.DataFrame(game_data)

Failed at /games/higurashi-when-they-cry-mei/
Failed at /games/cuisine-royale/
Failed at /games/faraway-2/
Failed at /games/nvidia-r-vr-funhouse/
Failed at /games/super-robot-wars-x-o/
Failed at /games/2nd-super-robot-wars-original-generation/
Failed at /games/toribash-violence-perfected/
Failed at /games/linda-3/
Failed at /games/ducktales-the-quest-for-gold/
Failed at /games/maze-war/


In [59]:
game_df_updated_ids = game_df[['url', 'game_id']]